In [0]:
--Rank stocks by daily return by day
with returns_data as (
  select ticker, trade_date,
  round((close-open)/open *100, 2) as returns
  from analytics.stock_prices
),

ranked as (
  select
  ticker, trade_date, returns,
  row_number() over(partition by trade_date order by returns desc) as rank
  from returns_data
)

select * 
from ranked
where rank <= 3 and returns >= 5
order by trade_date, rank;

--Assign row numbers to each stock's historical prices ordered by date'
select ticker,
trade_date,
close,
row_number() over(partition by ticker order by trade_date) as row_num
from analytics.stock_prices
order by ticker, trade_date;

--Get the top 3 highest volume days per stock
with volume_days as (
  select ticker, trade_date, volume,
  row_number() over(partition by ticker order by volume desc) as rank
  from analytics.stock_prices
)

select * 
from volume_days
where rank <= 3
order by ticker, rank;

--Get the top 3 lowest volume days per stock
with filtered as (
  select * 
  from analytics.stock_prices
  where volume != 0
),

volume_days as (
  select ticker, trade_date, volume,
  row_number() over(partition by ticker order by volume asc) as rank
  from filtered
)

select * 
from volume_days
where rank <= 3
order by ticker, rank;

--Identify the most volatile stock per month
with returns as (
  select ticker, trade_date,
  (close - lag(close) over(partition by ticker order by trade_date))
  / lag(close) over(partition by ticker order by trade_date) as daily_return
  from analytics.stock_prices
),

monthly_volume as (
  select ticker, year(trade_date) as yr, month(trade_date) as mo,
  round(stddev(daily_return) *100,2) as volatility
  from returns
  where daily_return is not null
  group by ticker, yr, mo
),

ranked as (
  select *,
  row_number() over (partition by yr, mo order by volatility desc) as rnk
  from monthly_volume
)

select *
from ranked
where rnk = 1
order by yr, mo;

--Robust Version of above query. Counts number of trading days to prevent partial months from distored volatility results.
WITH base AS (
    SELECT
        ticker,
        trade_date,
        close,
        LAG(close) OVER (
            PARTITION BY ticker
            ORDER BY trade_date
        ) AS prev_close
    FROM analytics.stock_prices
),

returns AS (
    SELECT
        ticker,
        trade_date,
        (close - prev_close) / prev_close AS daily_return
    FROM base
    WHERE prev_close IS NOT NULL
),

monthly_volatility AS (
    SELECT
        ticker,
        YEAR(trade_date) AS yr,
        MONTH(trade_date) AS mo,
        STDDEV(daily_return) * 100 AS volatility,
        COUNT(*) AS trading_days
    FROM returns
    GROUP BY ticker, yr, mo
    HAVING COUNT(*) >= 15
),

ranked AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY yr, mo
            ORDER BY volatility DESC
        ) AS rnk
    FROM monthly_volatility
)

SELECT *
FROM ranked
WHERE rnk = 1
ORDER BY yr, mo;
